<a href="https://colab.research.google.com/github/huylg/hcmaic_tqker/blob/master/HCMCAIC_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 2227 (delta 15), reused 16 (delta 7), pack-reused 2185
Receiving objects: 100% (2227/2227), 5.32 MiB | 4.29 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.


In [2]:
%cd yolov5

/content/yolov5


In [14]:
!cp '/content/drive/My Drive/hcmaic_tqker/videos/sample_01.mp4' /content/video

In [1]:
!cp -r '/content/drive/My Drive/aic-hcmc2020/videos' /content/video
!cp  '/content/drive/My Drive/output_yolov5/runs/exp1/weights/best.pt' /content/weights

^C


In [1]:
!pip install filterpy scikit-image lap


In [3]:
import sys
import os
import cv2
import argparse
from tqdm import tqdm
from detect import *
from sort import *
import json
import bb_polygon

In [4]:
def load_zone_anno(json_filename):
  """
  Load the json with ROI and MOI annotation.

  """
  with open(json_filename) as jsonfile:
    dd = json.load(jsonfile)
    polygon = [(int(x), int(y)) for x, y in dd['shapes'][0]['points']]
    paths = {}
    for it in dd['shapes'][1:]:
      kk = str(int(it['label'][-2:]))
      paths[kk] = [(int(x), int(y)) for x, y
              in it['points']]
  return polygon, paths


In [5]:
def check_bbox_intersect_polygon(polygon, bbox):
  """
  
  Args:
    polygon: List of points (x,y)
    bbox: A tuple (xmin, ymin, xmax, ymax)
  
  Returns:
    True if the bbox intersect the polygon
  """
  x1, y1, x2, y2 = bbox
  bb = [(x1,y1), (x2, y1), (x2,y2), (x1,y2)]
  return bb_polygon.is_bounding_box_intersect(bb, polygon)


In [18]:

#@markdown Your videos is stored in: 

input_video_dir='/content/video'
weight_path = '/content/weights/best.pt'
#@markdown  Frames extracted from videos will be stored in:
output_submit_dir='/content/output'
video_paths = []
for r, d, f in os.walk(input_video_dir):
    for file in f:
        if '.mp4' in file:
            video_paths.append(os.path.join(r, file))


for video_path in video_paths:
    print(video_path)


opt = Option(weight_path,conf_thres=0.4,output_path=output_submit_dir)
print(opt.output, opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size)
detector_tmp = detector(opt)
for video_path in video_paths:
  vid_cap = cv2.VideoCapture(video_path)
  num_frms, original_fps = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT)), vid_cap.get(cv2.CAP_PROP_FPS)
  polygon, paths = load_zone_anno('/content/video/sample_01.json')
  time_stride = 1
  for frm_id in (range(0, num_frms, time_stride)):
      vid_cap.set(cv2.CAP_PROP_POS_FRAMES, frm_id)
      _, im = vid_cap.read()
      cv2.imwrite('test.jpg', im)
      opt.source = 'test.jpg'
      pred = detector_tmp.detect(opt)
      dets = []
      for bbox, classId in pred:
        # tl =  round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
        # color =  [0,0,255]
        # # c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
        # cv2.rectangle(im, bbox[0], bbox[1], color, thickness=tl, lineType=cv2.LINE_AA)
        if check_bbox_intersect_polygon(polygon, (bbox[0][0], bbox[0][1], bbox[1][0], bbox[1][1])):
          # check if the bbox is in ROI
          # tl =  round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
          # color =  [255,0,0]
          # # c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
          # cv2.rectangle(im, bbox[0], bbox[1], color, thickness=tl, lineType=cv2.LINE_AA)
          # im = cv2.polylines(im,[polygon]],True,(0,0,255),thickness=3)
          
          # if label:
          #     tf = max(tl - 1, 1)  # font thickness
          #     t_size = cv2.getTextSize(classId, 0, fontScale=tl / 3, thickness=tf)[0]
          #     c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
          #     cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
          #     cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
          dets.append((frm_id, classId, bbox[0][0], bbox[0][1], bbox[1][0], bbox[1][1]))
        # else:
          # tl =  round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
          # color =  [255,255,0]
          # # c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
          # cv2.rectangle(im, bbox[0], bbox[1], color, thickness=tl, lineType=cv2.LINE_AA)
          # # if classId:
          # #   tf = max(tl - 1, 1)  # font thickness
          # #   t_size = cv2.getTextSize(classId, 0, fontScale=tl / 3, thickness=tf)[0]
          # #   c2 = bbox[0][0] + t_size[0], bbox[1][1] - t_size[1] - 3
          # #   cv2.rectangle(im, bbox[0], bbox[1], color, -1, cv2.LINE_AA)  # filled
          # #   cv2.putText(im, classId, (bbox[0][0], bbox[1][1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

          # # im = cv2.polylines(im,[polygon]],True,(0,0,255),thickness=3)
          # # cv2.imwrite('test_{}.jpg'.format(frm_id), im)
        # cv2.imwrite('/content/yolov5/output/test_image{}.jpg'.format(frm_id), im)
      print(dets)

Using CPU



/content/video/sample_01.mp4
/content/output  /content/weights/best.pt False False 640
Fusing layers... 


Model Summary: 284 layers, 8.84108e+07 parameters, 8.45317e+07 gradients


image 1/1 /content/yolov5/test.jpg: [(0, 0, 205, 373, 257, 501), (0, 0, 468, 160, 487, 195), (0, 3, 657, 141, 763, 293), (0, 1, 718, 356, 855, 591)]
image 1/1 /content/yolov5/test.jpg: [(1, 0, 206, 373, 257, 501), (1, 0, 468, 159, 486, 195), (1, 3, 657, 141, 763, 293), (1, 1, 718, 356, 855, 592)]
image 1/1 /content/yolov5/test.jpg: [(2, 0, 533, 129, 546, 164), (2, 3, 658, 131, 759, 296), (2, 0, 151, 418, 214, 600), (2, 0, 462, 162, 482, 197), (2, 1, 713, 347, 849, 471)]
image 1/1 /content/yolov5/test.jpg: [(3, 0, 371, 254, 393, 294), (3, 0, 316, 303, 350, 419), (3, 0, 457, 163, 478, 203), (3, 1, 710, 336, 838, 565)]
image 1/1 /content/yolov5/test.jpg: [(4, 0, 371, 254, 393, 294), (4, 0, 316, 303, 350, 419), (4, 0, 457, 163, 478, 203), (4, 1, 710, 336, 838, 565)]
image 1/1 /content/yolov5/test.jpg: [(5, 0, 455, 171, 476, 209), (5, 0, 529, 133, 544, 169), (5, 3, 653, 143, 755, 266), (5, 1, 710, 330, 832, 442)]
image 1/1 /content/yolov5/test.jpg: [(6, 0, 552, 126, 566, 164), (6, 0, 345, 2

KeyboardInterrupt: ignored